In [2]:
import pandas as pd
import sklearn as sklearn

df = pd.read_excel(r'file.xlsx', sheet_name='Sheet1')
print(df)


                                    User ID  applications  \
0      00007820-89cb-4c1d-9940-eb270d605a35             1   
1      00012b55-514c-421e-9c76-3300abbc1134             1   
2      000157c1-b6a3-4c86-82c7-9ec1bda3799a             1   
3      00020400-efab-4b10-8812-2a0aaf774841             1   
4      0002f737-0cda-48fb-91ed-533f3d0eab05             1   
...                                     ...           ...   
99995  fffb48e8-fc77-49fe-ab4e-e99a0829a788             1   
99996  fffb63e2-b1d6-4d73-b766-44ba50203c00             1   
99997  fffba42b-265f-4269-a6d5-c8db7e3bd430             1   
99998  fffd8e45-d5f8-4b06-b1a4-2d4bf527ee51             1   
99999  fffe836d-3176-4766-81c0-8812d1d9a0a6             1   

                         Reason  Loan_Amount  FICO_score Fico_Score_group  \
0      cover_an_unexpected_cost       100000         669             fair   
1       credit_card_refinancing        70000         594             fair   
2              home_improvement     

In [3]:
target_name = "Lender"
target = df[target_name]

data = df.drop(columns=[target_name])

from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)



In [4]:
apps = data_train
_ = model.fit(data_train, target_train)
model.predict(data_test)

array(['A', 'A', 'A', ..., 'A', 'A', 'A'], dtype=object)

In [5]:
apps["Lender"] = target_train

In [6]:
model.score(data_test, target_test)

0.62144

In [7]:
apps["Previous"] = df["Lender"]



In [9]:
apps["Lender"].head()

98980    C
69824    A
9928     C
75599    C
95621    B
Name: Lender, dtype: object

In [10]:
# Create a dictionary to map lender names to their respective values
lender_values = {'A': 250, 'B': 350, 'C': 150}

# Create a function to calculate the difference between two lenders
def calculate_difference(row):
    previous_lender = row['Previous']
    new_lender = row['Lender']
    previous_value = lender_values.get(previous_lender, 0) # Default to 0 if not found
    new_value = lender_values.get(new_lender, 0) # Default to 0 if not found
    return new_value - previous_value

# Create a new column "Difference" using the above function
apps['Difference'] = apps.apply(calculate_difference, axis=1)


In [11]:
apps["Difference"].head()

98980    0
69824    0
9928     0
75599    0
95621    0
Name: Difference, dtype: int64

In [12]:
grouped = apps.groupby("Lender")["Difference"].mean()

# Print the mean Difference for each lender
print("Mean Difference by Lender:")
print(grouped)

Mean Difference by Lender:
Lender
A    0.0
B    0.0
C    0.0
Name: Difference, dtype: float64
